###  Para encontrar minimos y máximos

Debemos resolver el siguente sistema: 
$$
    H_{r_n}(r_n - r_{r+1}) = J_{r_n}
$$
donde $H$ es la matriz hessiana, $J$ es el jacobiano y ambos están evaluados en el vector $r_n = (x_n,\ y_n)$

In [8]:
import sympy as sym
import numpy as np
# En esta celda se selecionan las funciones 
function = input("Introduce una función f(x, y) para calcular sus extremos. Utiliza las variables 'x' e 'y' y recuerda utilizar 'E' para el número e, expresar explicitamente que quieres multiplicar con '*', utilizar sin(x) en vez de sen(x) y utilizar paréntesis si utilizas funciones como seno y coseno \n")
symfunction = sym.sympify(function) # Esto es un casteo de una string a una expresión que sympy es capaz de entender 

F = sym.Matrix([symfunction]) # la matriz 1x1 de la función f

# Computamos la matriz que contiene las variables con las que diferenciaremos respecto de (cosas de sympy)
V = sym.Matrix([sym.sympify("x"), sym.sympify("y")])

# Computamos el Jacobiano
J = sym.transpose(F.jacobian(V)) # esta será nuestra matriz de coeficientes libres, Debe transponerse porque nos la da una matriz de 1x2

# Computamos el Hessiano
H = sym.hessian(F, (V))

error_dado = float(input("Introduce el error (Las raíces finales serán redondeadas en función del error dictado, Ej: error=0.0001 -> redondeo hasta la cuarta unidad): \n"))

Introduce una función f(x, y) para calcular sus extremos. Utiliza las variables 'x' e 'y' y recuerda utilizar 'E' para el número e, expresar explicitamente que quieres multiplicar con '*', utilizar sin(x) en vez de sen(x) y utilizar paréntesis si utilizas funciones como seno y coseno 
x^2+y^2
Introduce el error (Las raíces finales serán redondeadas en función del error dictado, Ej: error=0.0001 -> redondeo hasta la cuarta unidad): 
0.001


In [10]:
# En esta celda se hace la iteración
r0 = sym.Matrix([1, 1]) # Vector inicial, son matrices de sympy para que no halla errores
r_siguiente = sym.Matrix([0, 0]) # Vector siguente

#Primero lo hacemos con r0
# Actualizamos los valores de las matrices de coeficientes
J_sub = J.subs([[sym.sympify("x"), r0[0]], [sym.sympify("y"), r0[1]]])
H_sub = H.subs([[sym.sympify("x"), r0[0]], [sym.sympify("y"), r0[1]]])

solucion = H_sub.gauss_jordan_solve(J_sub)[0] # la fución debuelve dos matrices la primera tiene las soluciones 
# y la segunda tiene las incognitas en caso de que no pueda resolver el sistema completamente, como siempre va a poder 
# resolver el sistema solo necesitamos la primera enterada
# Actualizamos el valor de r_siguiente
r_siguiente = r0 - solucion

error_calculado = 1.0

iteracion = 0
while error_dado <= error_calculado or iteracion <= 10: # si hace demasiadas iteraciones parará el ciclo
    # Actualizamos los valores de las matrices de coeficientes
    J_sub = J.subs([[sym.sympify("x"), r_siguiente[0]], [sym.sympify("y"), r_siguiente[1]]])  # jacobiano sustituido
    H_sub = H.subs([[sym.sympify("x"), r_siguiente[0]], [sym.sympify("y"), r_siguiente[1]]]) # Hessiano sustituido
    solucion = H_sub.gauss_jordan_solve(J_sub)[0] # La matriz de soluciones
    
    # Actualizamos el valor de r_siguiete
    r_siguiente = r0 - solucion # Hacemos los dos a la vez
    
    # Actualizamos el error
    error_calculado = abs(max(r_siguiente.evalf()-r0.evalf()))

    # Actualizamos r0
    r0 = r_siguiente
    
    # Sumamos uno a la iteración 
    iteracion += 1
    print(f"Iteración: {iteracion}  Error: {error_calculado.evalf()}")

# Hacemos el redondeo de r_siguente con otro vector
x_siguiente = round(r_siguiente[0].evalf(), int(np.log10(error_dado**(-1))))
y_siguiente = round(r_siguiente[1].evalf(), int(np.log10(error_dado**(-1))))

F_sub = F.subs([[sym.sympify("x"), r_siguiente[0]], [sym.sympify("y"), r_siguiente[1]]])
print(f"Un extremo de la función es ({x_siguiente}, {y_siguiente}, {round(F_sub[0].evalf(), int(np.log10(error_dado**(-1))))}) ({iteracion} iteraciones)")

Iteración: 1  Error: 0
Iteración: 2  Error: 1.00000000000000
Iteración: 3  Error: 0
Iteración: 4  Error: 0
Iteración: 5  Error: 0
Iteración: 6  Error: 0
Iteración: 7  Error: 0
Iteración: 8  Error: 0
Iteración: 9  Error: 0
Iteración: 10  Error: 0
Iteración: 11  Error: 0
Un extremo de la función es (0, 0, 0) (11 iteraciones)


### Lo convertimos en una función

In [11]:
import sympy as sym
import numpy as np

def NewtonRaphson_2D_Optimos(f, tolerancia=0.001, max_iter=10, vectores_iniciales=[(1, 1), (-1, -1)]):
    """
    Lorem ipsum
    
    Input: 
        -f1: Debe ser una string. Una función que va a ser casteada a una función de Sympy. Utiliza una única variable 'x' 
        e 'y' para diferenciar la función respecto de ella y recuerda utilizar 'E' para el número e, expresar 
        explicitamente que quieres multiplicar con '*', utilizar sin(x) en vez de sen(x) y utilizar paréntesis si 
        utilizas funciones como seno y coseno
        
        -tolerancia: Para mejores resultados debe ser 10^n, (escrito como '0.0...01' con n ceros). Determina la precisión
        de las raíces obtenidas parando el proceso iterativo cuando el error calculado es menor que esta. Además en función
        de ella se redondea un set con las raices obtenidas (Ej: error=0.0001 -> redondeo hasta la cuarta unidad) ya que
        no tiene sentido que las raices obtenidas tengan más decimales que la tolerancia dada.
        
        -max_iter: Un número entero que representa el número máximo de iteraciones que la función calculará. Si se supera
        la función para los cálculos y devuelve la aproximación que tenga en ese momento
        
        -vectores_iniciales: Una lista de pares de numeros que representan estimaciones iniciales del punto de corte 
    Output:
        -
    """
    # Preparatorio
    
    # Casteamos las funciones a expresiones simbólicas de sympy y la ponemos en una matriz
    symf = sym.sympify(f)
    F_matriz = sym.Matrix([symf]) # la matriz 1x1 de la función f
    
    # Creamos el jacobiano y el hessiano de la función anterior
    V = sym.Matrix([sym.sympify("x"), sym.sympify("y")]) # Esta es la que contiene las variables en función de las 
    # cuales se toman las derivadas parciales, es necesáreo porque así está hecha la función .jacobian
    J = sym.transpose(F_matriz.jacobian(V))
    H = sym.hessian(F_matriz, (V))
    
    # Creamos la lista que contendrá los extremos encontrados y otra lista que tendrá si el vector conver o no
    vectores_solucion = list()
    vectores_que_convergen = list()
    
    # Iteramos sobre los vectores iniciales, aplicaremos el método a cada vector especificado para encontrar un mayor número
    # de extremos
    for vector_inicial in vectores_iniciales: # por cada uno de los vectores iniciales haremos el N-R
        # r0 es  el vector 'i' de los vectores iniciales y r_siguiente es por ahora nulo
        r0 = sym.Matrix(vector_inicial) 
        r_siguiente = sym.Matrix([0, 0])
        
        # Sustituimos r0 en la matriz f1_f2_matrix de las funciones y (r0) en el jacobiano
        J_en_r = J.subs([[sym.sympify("x"), r0[0]], [sym.sympify("y"), r0[1]]])
        H_en_r = H.subs([[sym.sympify("x"), r0[0]], [sym.sympify("y"), r0[1]]])
        
        # Calculamos la solución del sistema lineal con los coeficientes de la matriz del jacobiano en r0 y con los terminos 
        # indepencientes de la función en r0
        try:
            solucion = H_en_r.gauss_jordan_solve(J_en_r)[0] # la fución devuelve dos matrices la primera tiene las  
        # soluciones y la segunda tiene las incognitas en caso de que no pueda resolver el sistema completamente, como 
        # siempre va a poder resolver el sistema solo necesitamos la primera entera
        except ValueError:
            vectores_que_convergen.append(False)
            pass
        else:
            # Actualizamos el valor de r_siguiente
            r_siguiente = r0 - solucion

            # Declaramos 'error_calculado' que es la variable que guardará el error en cada iteración y 'iteración' que 
            # guardará porqué iteración vamos
            error_calculado = 1.0
            iteracion = 0

            # Bucle iterativo de newton raphson 
            while tolerancia <= error_calculado or iteracion <= max_iter: # si hace demasiadas iteraciones parará el ciclo
                J_en_r = J.subs([[sym.sympify("x"), r_siguiente[0]], [sym.sympify("y"), r_siguiente[1]]])  # jacobiano sustituido
                H_en_r = H.subs([[sym.sympify("x"), r_siguiente[0]], [sym.sympify("y"), r_siguiente[1]]]) # Hessiano sustituido
                solucion = H_en_r.gauss_jordan_solve(J_en_r)[0] # La matriz de soluciones

                # Actualizamos el valor de r_siguiente
                r_siguiente = r0 - solucion # Hacemos los dos a la vez

                # Actualizamos el error
                error_calculado = abs(max([(r_siguiente[0]-r0[0]), (r_siguiente[1]-r0[1])]))

                # Actualizamos r0
                r0 = r_siguiente

                # Sumamos uno a la iteración 
                iteracion += 1
                #print(f"Iteración: {iteracion}  Error: {error_calculado.evalf()}")

            # Hacemos el redondeo de r_siguente con otro vector
            x_siguiente = round(r_siguiente[0].evalf(), int(np.log10(tolerancia**(-1))))
            y_siguiente = round(r_siguiente[1].evalf(), int(np.log10(tolerancia**(-1))))

            F_matriz_en_r = F_matriz.subs([[sym.sympify("x"), r_siguiente[0].evalf()], [sym.sympify("y"), r_siguiente[1].evalf()]])
            z_resultante = round(F_matriz_en_r[0].evalf(), int(np.log10(tolerancia**(-1))))

            # Añadimos el vector encontrado a la lista vectores_solucion
            vectores_solucion.append([x_siguiente, y_siguiente, z_resultante]) 
            vectores_que_convergen.append(True)
    return vectores_solucion, vectores_que_convergen

In [13]:
print(NewtonRaphson_2D_Optimos("x^3 +y^2", tolerancia=0.001, max_iter=10, vectores_iniciales=[(1, 1)]))

Iteración: 1  Error: 0
Iteración: 2  Error: 0.375000000000000
Iteración: 3  Error: 0
Iteración: 4  Error: 0
Iteración: 5  Error: 0
Iteración: 6  Error: 0
Iteración: 7  Error: 0
Iteración: 8  Error: 0
Iteración: 9  Error: 0
Iteración: 10  Error: 0
Iteración: 11  Error: 0
([[0.001, 0, 0.0]], [True])


In [14]:
from tkinter import *

def Pasar_parametros():
    funcion = entrada_funcion.get()
    tolerancia = float(entrada_tolerancia.get()) 
    max_iter = int(entrada_max_iter.get())   
    vector_inicial = [(float(entrada_vx.get()), float(entrada_vy.get()))]
    
    extremos = NewtonRaphson_2D_Optimos(f=funcion, tolerancia=tolerancia, max_iter=max_iter, vectores_iniciales=vector_inicial)[0]
    etiqueta_extremos["text"] = f"Se ha encontrado el extremo: {extremos}"
    print(extremos)
ventana = Tk() 
ventana.title("Encontrador de extremos mediante Newton-Raphson")

# hacemos un marco con todas las imputs
marco_inputs = LabelFrame(ventana, text="Entrada", padx=10, pady=10) 

# Definimos todo
etiqueta_funcion = Label(marco_inputs, text="Introduce la función: ")
etiqueta_tolerancia = Label(marco_inputs, text="Introduce la tolerancia: ")
etiqueta_max_iter = Label(marco_inputs, text="Introduce el número máximo de iteraciones: ")
etiqueta_v_inicial = Label(marco_inputs, text="Introduce el vector inicial (x arriva): ")

entrada_funcion = Entry(marco_inputs, width=35, borderwidth=5)
entrada_tolerancia = Entry(marco_inputs, width=35, borderwidth=5)
entrada_max_iter = Entry(marco_inputs, width=35, borderwidth=5)
marco_vector_inicial = LabelFrame(marco_inputs)

entrada_vx = Entry(marco_vector_inicial)
entrada_vy = Entry(marco_vector_inicial)

entrada_vx.pack()
entrada_vy.pack()

# Ponemos todo con .gird. Las etiquetas estarán pegadas a la izq.
etiqueta_funcion.grid(row=0, column=0, sticky=W)
etiqueta_tolerancia.grid(row=1, column=0, sticky=W)
etiqueta_max_iter.grid(row=2, column=0, sticky=W)

entrada_funcion.grid(row=0, column=1)
entrada_tolerancia.grid(row=1, column=1)
entrada_max_iter.grid(row=2, column=1)

etiqueta_v_inicial.grid(row=3, column=0)
marco_vector_inicial.grid(row=3, column=1)

boton_calc_extremos = Button(marco_inputs, text="Calcular con parámetros actuales", command=Pasar_parametros)

boton_calc_extremos.grid(row=4, column=0, columnspan=1)

marco_inputs.pack()

#Hacemos un marco con todas las outputs
marco_outputs = LabelFrame(ventana, text="Salida", padx=10, pady=10) 

etiqueta_extremos = Label(marco_outputs, text="Esperando a una función")

etiqueta_extremos.grid(row=0, column=0)

marco_outputs.pack()

ventana.mainloop()

Iteración: 1  Error: 0
Iteración: 2  Error: 1.00000000000000
Iteración: 3  Error: 0
Iteración: 4  Error: 0
Iteración: 5  Error: 0
Iteración: 6  Error: 0
[[0, 0, 0]]
Iteración: 1  Error: 0
Iteración: 2  Error: 1.00000000000000
Iteración: 3  Error: 0
Iteración: 4  Error: 0
Iteración: 5  Error: 0
Iteración: 6  Error: 0
[[0, 0, 0]]


In [15]:
#ignorar esto

marco_vectores_iniciales = LabelFrame(marco_inputs, text="Vectores iniciales (x, y)", padx=10, pady=10)

# Definimos

marco_v1 = LabelFrame(marco_vectores_iniciales)
#marco_v2 = LabelFrame(marco_vectores_iniciales)
#marco_v3 = LabelFrame(marco_vectores_iniciales)
#marco_v4 = LabelFrame(marco_vectores_iniciales)

etiqueta_vector1 = Label(marco_v1, text="Vector 1: ")
#etiqueta_vector2 = Label(marco_v2, text="Vector 2: ")
#etiqueta_vector3 = Label(marco_v3, text="Vector 3: ")
#etiqueta_vector4 = Label(marco_v4, text="Vector 4: ")

entrada_vector1 = Entry(marco_v1, width=10, borderwidth=1)
#entrada_vector2 = Entry(marco_v2, width=10, borderwidth=1)
#entrada_vector3 = Entry(marco_v3, width=10, borderwidth=1)
#entrada_vector4 = Entry(marco_v4, width=10, borderwidth=1)

# Lo pasamos por pantalla

etiqueta_vector1.pack()
#etiqueta_vector2.pack()
#etiqueta_vector3.pack()
#etiqueta_vector4.pack()

entrada_vector1.pack()
#entrada_vector2.pack()
#entrada_vector3.pack()
#entrada_vector4.pack()

marco_v1.grid(row=0, column=0)
#marco_v2.grid(row=0, column=1)
#marco_v3.grid(row=0, column=2)
#marco_v4.grid(row=0, column=3)

marco_vectores_iniciales.grid(row=3, column=0) 

TclError: can't invoke "labelframe" command: application has been destroyed

In [ ]:
import sympy as sym
import numpy as np
from tkinter import *


def NewtonRaphson_2D_Optimos(f, tolerancia=0.001, max_iter=10, vectores_iniciales=[(1, 1), (-1, -1)]):
    """
    Lorem ipsum
    
    Input: 
        -f1: Debe ser una string. Una función que va a ser casteada a una función de Sympy. Utiliza una única variable 'x' 
        e 'y' para diferenciar la función respecto de ella y recuerda utilizar 'E' para el número e, expresar 
        explicitamente que quieres multiplicar con '*', utilizar sin(x) en vez de sen(x) y utilizar paréntesis si 
        utilizas funciones como seno y coseno
        
        -tolerancia: Para mejores resultados debe ser 10^n, (escrito como '0.0...01' con n ceros). Determina la precisión
        de las raíces obtenidas parando el proceso iterativo cuando el error calculado es menor que esta. Además en función
        de ella se redondea un set con las raices obtenidas (Ej: error=0.0001 -> redondeo hasta la cuarta unidad) ya que
        no tiene sentido que las raices obtenidas tengan más decimales que la tolerancia dada.
        
        -max_iter: Un número entero que representa el número máximo de iteraciones que la función calculará. Si se supera
        la función para los cálculos y devuelve la aproximación que tenga en ese momento
        
        -vectores_iniciales: Una lista de pares de numeros que representan estimaciones iniciales del punto de corte 
    Output:
        -
    """
    # Preparatorio
    
    # Casteamos las funciones a expresiones simbólicas de sympy y la ponemos en una matriz
    symf = sym.sympify(f)
    F_matriz = sym.Matrix([symf]) # la matriz 1x1 de la función f
    
    # Creamos el jacobiano y el hessiano de la función anterior
    V = sym.Matrix([sym.sympify("x"), sym.sympify("y")]) # Esta es la que contiene las variables en función de las 
    # cuales se toman las derivadas parciales, es necesáreo porque así está hecha la función .jacobian
    J = sym.transpose(F_matriz.jacobian(V))
    H = sym.hessian(F_matriz, (V))
    
    # Creamos la lista que contendrá los extremos encontrados y otra lista que tendrá si el vector conver o no
    vectores_solucion = list()
    vectores_que_convergen = list()
    
    # Iteramos sobre los vectores iniciales, aplicaremos el método a cada vector especificado para encontrar un mayor número
    # de extremos
    for vector_inicial in vectores_iniciales: # por cada uno de los vectores iniciales haremos el N-R
        # r0 es  el vector 'i' de los vectores iniciales y r_siguiente es por ahora nulo
        r0 = sym.Matrix(vector_inicial) 
        r_siguiente = sym.Matrix([0, 0])
        
        # Sustituimos r0 en la matriz f1_f2_matrix de las funciones y (r0) en el jacobiano
        J_en_r = J.subs([[sym.sympify("x"), r0[0]], [sym.sympify("y"), r0[1]]])
        H_en_r = H.subs([[sym.sympify("x"), r0[0]], [sym.sympify("y"), r0[1]]])
        
        # Calculamos la solución del sistema lineal con los coeficientes de la matriz del jacobiano en r0 y con los terminos 
        # indepencientes de la función en r0
        try:
            solucion = H_en_r.gauss_jordan_solve(J_en_r)[0] # la fución devuelve dos matrices la primera tiene las  
        # soluciones y la segunda tiene las incognitas en caso de que no pueda resolver el sistema completamente, como 
        # siempre va a poder resolver el sistema solo necesitamos la primera entera
        except ValueError:
            vectores_que_convergen.append(False)
            pass
        else:
            # Actualizamos el valor de r_siguiente
            r_siguiente = r0 - solucion

            # Declaramos 'error_calculado' que es la variable que guardará el error en cada iteración y 'iteración' que 
            # guardará porqué iteración vamos
            error_calculado = 1.0
            iteracion = 0

            # Bucle iterativo de newton raphson 
            while tolerancia <= error_calculado or iteracion <= max_iter: # si hace demasiadas iteraciones parará el ciclo
                J_en_r = J.subs([[sym.sympify("x"), r_siguiente[0]], [sym.sympify("y"), r_siguiente[1]]])  # jacobiano sustituido
                H_en_r = H.subs([[sym.sympify("x"), r_siguiente[0]], [sym.sympify("y"), r_siguiente[1]]]) # Hessiano sustituido
                solucion = H_en_r.gauss_jordan_solve(J_en_r)[0] # La matriz de soluciones

                # Actualizamos el valor de r_siguiente
                r_siguiente = r0 - solucion # Hacemos los dos a la vez

                # Actualizamos el error
                error_calculado = abs(max([(r_siguiente[0]-r0[0]), (r_siguiente[1]-r0[1])]))

                # Actualizamos r0
                r0 = r_siguiente

                # Sumamos uno a la iteración 
                iteracion += 1
                #print(f"Iteración: {iteracion}  Error: {error_calculado.evalf()}")

            # Hacemos el redondeo de r_siguente con otro vector
            x_siguiente = round(r_siguiente[0].evalf(), int(np.log10(tolerancia**(-1))))
            y_siguiente = round(r_siguiente[1].evalf(), int(np.log10(tolerancia**(-1))))

            F_matriz_en_r = F_matriz.subs([[sym.sympify("x"), r_siguiente[0].evalf()], [sym.sympify("y"), r_siguiente[1].evalf()]])
            z_resultante = round(F_matriz_en_r[0].evalf(), int(np.log10(tolerancia**(-1))))

            # Añadimos el vector encontrado a la lista vectores_solucion
            vectores_solucion.append([x_siguiente, y_siguiente, z_resultante]) 
            vectores_que_convergen.append(True)
    return vectores_solucion, vectores_que_convergen

def Pasar_parametros():
    funcion = entrada_funcion.get()
    tolerancia = float(entrada_tolerancia.get()) 
    max_iter = int(entrada_max_iter.get())   
    vector_inicial = [(float(entrada_vx.get()), float(entrada_vy.get()))]
    
    extremos = NewtonRaphson_2D_Optimos(f=funcion, tolerancia=tolerancia, max_iter=max_iter, vectores_iniciales=vector_inicial)[0]
    etiqueta_extremos["text"] = f"Se ha encontrado el extremo: {extremos}"
    print(extremos)
ventana = Tk() 
ventana.title("Encontrador de extremos mediante Newton-Raphson")

# hacemos un marco con todas las imputs
marco_inputs = LabelFrame(ventana, text="Entrada", padx=10, pady=10) 

# Definimos todo
etiqueta_funcion = Label(marco_inputs, text="Introduce la función: ")
etiqueta_tolerancia = Label(marco_inputs, text="Introduce la tolerancia: ")
etiqueta_max_iter = Label(marco_inputs, text="Introduce el número máximo de iteraciones: ")
etiqueta_v_inicial = Label(marco_inputs, text="Introduce el vector inicial (x arriva): ")

entrada_funcion = Entry(marco_inputs, width=35, borderwidth=5)
entrada_tolerancia = Entry(marco_inputs, width=35, borderwidth=5)
entrada_max_iter = Entry(marco_inputs, width=35, borderwidth=5)
marco_vector_inicial = LabelFrame(marco_inputs)

entrada_vx = Entry(marco_vector_inicial)
entrada_vy = Entry(marco_vector_inicial)

entrada_vx.pack()
entrada_vy.pack()

# Ponemos todo con .gird. Las etiquetas estarán pegadas a la izq.
etiqueta_funcion.grid(row=0, column=0, sticky=W)
etiqueta_tolerancia.grid(row=1, column=0, sticky=W)
etiqueta_max_iter.grid(row=2, column=0, sticky=W)

entrada_funcion.grid(row=0, column=1)
entrada_tolerancia.grid(row=1, column=1)
entrada_max_iter.grid(row=2, column=1)

etiqueta_v_inicial.grid(row=3, column=0)
marco_vector_inicial.grid(row=3, column=1)

boton_calc_extremos = Button(marco_inputs, text="Calcular con parámetros actuales", command=Pasar_parametros)

boton_calc_extremos.grid(row=4, column=0, columnspan=1)

marco_inputs.pack()

#Hacemos un marco con todas las outputs
marco_outputs = LabelFrame(ventana, text="Salida", padx=10, pady=10) 

etiqueta_extremos = Label(marco_outputs, text="Esperando a una función")

etiqueta_extremos.grid(row=0, column=0)

marco_outputs.pack()

ventana.mainloop()